In [ ]:
#!/usr/bin/env python3

In [2]:
import numpy as np 
import time 
from datetime import timedelta
import tensorflow as tf
from readImages import *
from build_tensorflow_graph import *

In [3]:
#
## ---------- VAE Architecture ----------
#

# Convolution Layer 1
numFilters1 = 64
conv1 = CNN_Architecture(numFilters = numFilters1, 
                         filterSize = (3, 3), 
                         strides = 1, 
                         toPadding = True, 
                         useReLU = True, 
                         numInputChannels = 3, 
                         maxPoolingSize=(2, 2))
# Convolution Layer 2
numFilters2 = 128
conv1 = CNN_Architecture(numFilters = , numFilters2
                         filterSize = (3, 3), 
                         strides = 1, 
                         toPadding = True, 
                         useReLU = True, 
                         numInputChannels = numFilters1, 
                         maxPoolingSize=(2, 2))
# Convolution Layer 3
numFilters2 = 128
conv1 = CNN_Architecture(numFilters = , numFilters3
                         filterSize = (3, 3), 
                         strides = 1, 
                         toPadding = True, 
                         useReLU = True, 
                         numInputChannels = numFilters2, 
                         maxPoolingSize=(2, 2))
# Fully Connected 1
fc1_size = 2048
# Fully Connected 2
fc2_size = 512
# Lattern Code 
z_dim = 32

In [4]:
#  
#    input layer: 64 * 64 * 3 = 12288
#    conv1 layer: 32 * 32 * 64 = 65536
#    conv2 layer: 16 * 16 * 128 = 32768
#    conv3 layer: 8 * 8 * 128 = 8192
#    fc1_size = 2048
#    fc2_size = 512
#    z_dim = 32

In [ ]:
class VAE:
    def __init__(self):
        raise NotImplementedError
    def encoder(self):
        raise NotImplementedError
    def decoder(self):
        raise NotImplementedError
    def buildModel(self):
        raise NotImplementedError
    def train(self):
        raise NotImplementedError
    def generateFakeImages(self):
        raise NotImplementedError
    